
   
$$\LARGE \mathbf{Comoving~Frames}$$

$$\normalsize \mathrm{Christian~Knigge}$$

$$\normalsize \mathrm{July~2020}$$
    

# Background

This document is intended as a purely pedagogical treatment of some problems that have arisen in the context of our implementation of special relativistic effects into *python*.

Note that when I say "pedagogical", I really mean "pedagogical *for me*". In fact, one of the first things I realized when I started looking into these issues is that I am extremely poorly equipped to deal with them -- relativity was very sketchily covered in my undergraduate syllabus (I blame Southampton), and I have simply not had any reason to (re)learn things until now. 

As a result, my efforts here are probably going to be mostly about correctly *interpreting* and *using* the results of others, rather than deriving things from first principles. And just to give myself an excuse for (a) not trying a "first principles" approach, and (b) almost certainly screwing things up, here is the final paragraph from the intro to Section 6 of Castor's book on radiation hydrodynamics (this is the section on SR):

"*This material is admittedly complicated. The reader is encouraged to find the other references, including the key paper of L. H. Thomas (1930) and the report by Fraser (1966). This topic is Chapter 7 in Mihalas and Mihalas (1984), and the present discussion also draws on Castor (1972).*"
    
If *Castor* finds something complicated, it probably really is....

this document was mostly written by CK. JM has added the odd note. 

# Some definitions

## Basics

* $\beta = v /c$

* $\gamma  = \left(1 - \beta^2\right)^{-1/2}$

* *Proper* time is time defined in the *comoving* frame.


## Lorentz transformations in vector form

### Standard notation

The Wikipedia page on Lorentz transformations

https://en.wikipedia.org/wiki/Lorentz_transformation#Vector_transformations

helpfully provides them in vector form (this is actually kind of hard to find elsewhere). As they are written on that page, note that (in our notation) the prime frame refers to the cmf frame, and the unprimed frame to the observer frame. That follows because the page notes explicitly that an observer in the unprimed frame sees the primed frame as moving with velocity $\mathbf{v}$ (whereas an observer in the primed frame sees the unprimed frame moving at $-\mathbf{v}$. In our notation, a cell moves at velocity $\mathbf{v}$ in the observer frame, so unprimed = observer.

For reference then, and converting to our notation, the transformation for the time coordinate is


\begin{equation}
t_{cmf} = \gamma \left[ t_{obs} - \frac{\mathbf{v} \cdot \mathbf{r}_{obs}}{c^2} \right],
\tag{1}
\end{equation}

where $\mathbf{\hat{v}}$ is the unit velocity vector of the flow, as measured in the observer frame. And the transformation for the position vector is


\begin{equation}
\mathbf{r}_{cmf} = \mathbf{r}_{obs} + (\gamma - 1)(\mathbf{r}_{obs} \cdot \mathbf{\hat{v}})\mathbf{\hat{v}} - \gamma \delta t_{obs} \mathbf{v}.
\tag{2}
\end{equation}

### *Pythonic* notation

Looking ahead to how we will actually use those quantities, we can also rewrite them slightly. We will eventually want to apply them to photon trajectories ($\mathbf{\delta s}_{cmf}$ and $\mathbf{\delta s}_{obs}$) inside a cell. So we define both the observer and cmf frames to have their origin at the starting point of the photon's trajectory. We also define $t_{obs} = t_{cmf} = 0$ when the photon starts on its trajectory. We can then simply replace $t \rightarrow \delta t$ and $\mathbf{r} \rightarrow \mathbf{\delta s}$, giving


\begin{equation}
\delta t_{cmf} = \gamma \left[ \delta t_{obs} - \frac{\mathbf{v} \cdot \mathbf{\delta s_{obs}}}{c^2} \right]
\tag{3}
\end{equation}


\begin{equation}
\mathbf{\delta s}_{cmf} = \mathbf{\delta s}_{obs} + (\gamma - 1)(\mathbf{\delta s}_{obs} \cdot \mathbf{\hat{v}})\mathbf{\hat{v}} - \gamma \delta t_{obs} \mathbf{v}
\tag{4}
\end{equation}


# The problem

Ultimately, the issue we are trying to address is how to correctly formulate Monte Carlo estimators for things like photo-ionization rates in cases where SR effects cannot be ignore.

Note that actually *all* estimators probably need to be thought about -- i.e. ionization rates, recombination rates, heating rates, cooling rates, SEDs. I.e. all of these quantities are likely to be different in comoving and observer frames.

## The tension between observer and comoving frames

The fundamental tension between the two frames is something that is clear from Stuart's notes:

**All the standard atomic physics and equations describing matter-radiation interactions are *only* valid in the comoving frame.**

**BUT**

**The space-time grid on which we carry out our computations is defined in the *observer* frame.** Note that by "space-time grid" I mean our spatial grid and the time-steps we implicitly [or, in the case of reverberation mapping, explicitly] consider when we carry out calculations.

# An attempt at a solution

## Photoionization in the comoving frame

In order to fix ideas, I'm going to follow Stuart and consider the photo-ionization rate as a concrete example. My starting point will be his Equation 4, which I think is uncontroversially correct:

\begin{equation}
\substack{\text{ionizations}\\\\\text{per unit}\\\text{proper time}} = \frac{N_{a,cmf}}{\Delta V_{cmf}\Delta t_{cmf}} \sum_i^{\Delta V_{cmf}\Delta t_{cmf}}\frac{\epsilon_{cmf,i}}{h\nu_{cmf,i}} \sigma(\nu_{cmf,i}) \delta s_{cmf,i}
\tag{5}
\end{equation}

This tells us how many ionizations will happen per unit proper (cmf) time in the *comoving* spatial volume element $\Delta V_{cmf}$. 

The sum is over all photons found inside the *comoving* space-time volume element $\Delta V_{cmf} \Delta t_{cmf}$.

Note that I have intentionally omitted the subscript "cmf" from $\sigma$ to emphasize the fact that the cross-section here is the standard atomic physics quantity we know and love. *All* atomic physics quantities like this are *defined* in the comoving frame.

$N_{a,cmf}$ is the number of (relevant) *atoms* found in the *comoving* spatial volume element $\Delta V_{cmf}$. We'll assume for the moment that this number is fixed in (comoving) time, i.e. the flow is in a steady-state. 

I have used a different (lower case) "delta" symbol for $\delta s_{cmf}$, because $\delta s_{cmf}$ is, by construction, always less than or equal to the size of the cmf volume $\Delta V_{cmf}$, measured along the photon's direction, $\mathbf{\hat{n}}_{cmf}$. (I.e. all the path lengths we count must be *inside* the volume under consideration.)

An important point is that the units here are "ionizations per unit proper time" -- they are *not* "ionizations per unit proper time and comoving volume". Even though $\Delta V_{cmf}$ appears in the denominator, this "per-volume" bit is cancelled by the $\sigma \,\delta s$ term in the summation.

Partly because this last point could easily cause confusion -- and also following Stuart -- let's switch to working in terms of *atom density*. So we define $n_{a,cmf} = N_{a,cmf} / \Delta V_{cmf}$ as the comoving frame atom density and rewrite Equation 1 as 

\begin{equation}
\substack{\text{ionizations}\\\\\text{per unit}\\\text{proper time}} = \frac{n_{a,cmf}}{\Delta t_{cmf}} \sum_i^{\Delta V_{cmf}\Delta t_{cmf}} \frac{\epsilon_{cmf,i}}{h\nu_{cmf,i}} \sigma(\nu_{cmf,i}) \delta s_{cmf,i},
\label{eq2}
\tag{6}
\end{equation}
where $n_{a,cmf} = N_{a,cmf} / \Delta V_{cmf}$ is the comoving frame atom density.

Finally, I'm going to do something a little different. Suppose I want to define the ionization rate in a way that is Lorentz invariant. I can do that by noting that $\Delta V \Delta t$ is Lorentz invariant, so if I define the number of ionizations *per space-time volume element* (instead of per unit time), that number must be the same in all frames. So I'll define

\begin{equation}
\substack{\text{ionizations}\\\\\text{per unit}\\\text{cmf space-time volume}} = \frac{n_{a,cmf}}{\Delta V_{cmf} \Delta t_{cmf}} \sum_i^{\Delta V_{cmf}\Delta t_{cmf}} \frac{\epsilon_{cmf,i}}{h\nu_{cmf,i}} \sigma(\nu_{cmf,i}) \delta s_{cmf,i}.
\tag{7}
\end{equation}

## Photoionization in the observer frame

Now let's consider the observer frame. The equivalent to Equation 7 in the observer frame is

\begin{equation}
\substack{\text{ionizations}\\\\\text{per unit}\\\text{observer space-time volume}} = 
\frac{n_{a,obs}}{\Delta V_{obs}\Delta t_{obs}} \sum_i^{\Delta V_{obs}\Delta t_{obs}}\frac{\epsilon_{obs,i}}{h\nu_{obs,i}} \sigma_{obs,i}(\nu_{cmf,i}) \delta s_{obs,i}
\tag{8a}
\end{equation}

Note that *everything* here is in the observer frame, *except* the photon frequency entering into the cross-section, since the cross-section clearly must be evaluated at the comoving photon frequency.

**It's important to note here that, unlike Equation 7, which is a statement of physics, Equation 8 is really just an "Ansatz" -- it represents a choice.** 

What does that mean? It means we've *decided* that we'd like to have an equation of exactly the same form as Equation 7 to also hold when we use observer frame quantities. As we'll see in a minute, with one exception, all of the quantities on the right-hand side are either Lorentz invariant or purely geometric ($\delta s)$. The one exception is $\sigma_{obs}$, so effectively Equation 8 represents the *definition* of what we must mean by an "observer frame cross-section".

An equally valid alternative choice would be to argue that there is no such thing as a separate "observer frame cross-section". We can absolutely do this, so long as we then allow for an extra "frame transformation factor" -- let's call it $K_{obs}$ -- on the right-hand side. That is, we could equally make the Ansatz that 

\begin{equation}
\substack{\text{ionizations}\\\\\text{per unit}\\\text{observer space-time volume}} = 
\frac{n_{a,obs}}{\Delta V_{obs}\Delta t_{obs}} \sum_i^{\Delta V_{obs}\Delta t_{obs}}\frac{\epsilon_{obs,i}}{h\nu_{obs,i}} K_{obs,i}  \sigma(\nu_{cmf,i}) \delta s_{obs,i}
\tag{8b}
\end{equation}

Since we'll find below that $\sigma_{obs,i}$ (or, equivalently, $K_{obs,i}$) are direction dependent, I think Equation 8b is more intuitive -- or at least less confusing -- for at least some of us. I'll therefore default to this form from now on.

Note also that the *direction* of the photons in the observer frame is not the same as in the comoving frame, so we'll define the observer-frame direction vector $\mathbf{\hat{n}}_{obs,i}$.

Because we have written the ionization rate now as "ionizations per unit space-time volume element", the left-hand sides of both Equations 7 and 8a/8b must be equal -- that number is a Lorentz invariant. (That was the point of doing this.) 

## Relating the comoving and observer frame pictures

We begin by making an explicit choice about our space-time volume elements. Remember (from Stuart's original derivation) that our sum in Equation 4 is over photon bundles that pass through the cmf space-time volume element we are considering. The important point (to me) here is that it's up to us to decide *which* element this is. So I'm going to decide that the relevant cmf space-time volume is that which corresponds *exactly* to the observer-frame space-time volume element $\Delta V_{obs} \Delta t_{obs}$, where $\Delta V_{obs}$ can now be an actual (square, say) "cell" in a computational grid. 

This choice means two things:

* First, we can immediately use the Lorentz invariance of space-time volume elements to say that  $\Delta V_{cmf} \Delta t_{cmf} = \Delta V_{obs} \Delta t_{obs} = \Delta V \Delta t$.



* Second, we are now explicitly talking about the *same* photon bundles in both cases, so their number, $N_{\gamma} = \frac{\epsilon_{cmf,i}}{h\nu_{cmf,i}} = \frac{\epsilon_{obs,i}}{h\nu_{obs,i}}$ is also the same in both frames.


So let's set our two equations equal to each other and cancel things that are manifestly the same:


\begin{equation}
\substack{\text{ionizations}\\\\\text{per unit}\\\text{space-time volume}} = \\
n_{a,cmf} \sum_i^{\Delta V\Delta t} \sigma(\nu_{cmf,i}) \delta s_{cmf,i} = 
n_{a,obs} \sum_i^{\Delta V\Delta t} K_{obs,i} \sigma(\nu_{cmf,i}) \delta s_{obs,i}
\tag{9}
\end{equation}

What else do we know? Well, densities in the comoving and observer frames are not the same. Specifically, for number densities, we have $n_{a,obs} = \gamma n_{a,cmf}$ (mihalas-1.pdf; p146). This just follows from Lorentz contraction -- the observer thinks the cmf is moving and thus that all lengths along the direction of motion are compressed by $1/\gamma$. Since density is number / volume, and since lengths along directions perpendicular to the direction of motion are unchanged, she therefore measures a density that is *higher* by a factor $\gamma$ than an observer in the cmf.

Also, given that we've made the choice that we're considering the *same* photon bundles in both frames, we may as well just consider a single one of them, so that we can drop the summation.

These considerations lead us to:

\begin{equation}
\sigma(\nu_{cmf}) \delta s_{cmf} = 
\gamma K_{obs} \sigma(\nu_{cmf}) \delta s_{obs}
\tag{10}
\end{equation}

Now we just need to apply a Lorentz transformation to the $\delta s$, which will then tell us what $K_{obs}$ is.

So, let's check how $\delta s$ transforms. In the observer frame, the photon travels distance $\delta s_{obs} = c \delta t_{obs}$, while in the comoving frame, the photon travels distance $\delta s_{cmf} = c \delta t_{cmf}$.  Note that $\delta t_{obs} \neq \Delta t_{obs}$, in general.

As discussed already above (in the Section on vector Lorentz transformations), we now define both frames to have their origin at the starting point of the photon's trajectory at $t_{obs,0} = t_{cmf,0} = 0$.  The observer and comoving $\delta t$ then just transform via an ordinary Lorentz transformation for time coordinates:

\begin{equation}
\delta t_{cmf} = \gamma \left[ \delta t_{obs} - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs}) \delta s_{obs}}{c^2} \right]
\tag{11}
\end{equation}

We can simplify by substituting  $\delta s_{obs} = c \delta t_{obs}$:

\begin{equation}
\delta t_{cmf} = \gamma \delta t_{obs} \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right]
\tag{12}
\end{equation}


Multiplying both sides by $c$ then gives us the desired transformation:


\begin{equation}
\delta s_{cmf} = \gamma \delta s_{obs} \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right]
\tag{13}
\end{equation}

This agrees to within the factor $\gamma$ with Leon Lucy's expression in his 2005 paper, which he says is accurate to O(v/c). It also agrees with Stuart's expression.

We can now substitute this back into Equation 10 and cancel stuff, giving:


\begin{equation}
\sigma(\nu_{cmf})  \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right] = 
 K_{obs}\sigma(\nu_{cmf})
\tag{14}
\end{equation}


and thus

\begin{equation}
K_{obs} = \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right] 
\tag{15}
\end{equation}



For reference, in the language of Equation 8a, the "observer frame cross-section" has to be


\begin{equation}
\sigma_{obs} = \sigma \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right] \
\tag{16}
\end{equation}

Plugging all this back into Equation 8a/8b, we get our final estimate for the ionization rate estimator in terms of things we actually know. 

\begin{equation}
\substack{\text{ionizations}\\\\\text{per unit}\\\text{invariant space-time volume}} = 
\frac{n_{a,obs}}{\Delta V_{obs}\Delta t_{obs}} \sum_i^{\Delta V_{obs}\Delta t_{obs}}\frac{\epsilon_{obs,i}}{h\nu_{obs,i}} \sigma(\nu_{cmf,i}) \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs,i})}{c} \right]  \delta s_{obs,i}
\tag{17}
\end{equation}




## How would this actually work in *Python*?

### The photo-ionization rate estimate for macro-atoms

The first thing to note is that, in *Python*, photon bundles carry luminosity, not energy, i.e. the (observer frame) weight of a photon bundle is defined as

$$w_{obs,i} = \frac{\epsilon_{obs,i}} {\Delta t_{obs}}$$

In *Python*, the macro-atom estimator for b-f processes is updated in function bf_estimators_increment within estimators.c

That function is (currently) passed the wind pointer, the photon pointer and ds (in the observer frame).

Optical depth can be written in two equivalent ways:

$$\tau = \rho \,\, \kappa \,\, ds = n \,\, \sigma \,\, ds$$

Here, $\sigma$ is the cross-section (as usual), and $\kappa$ is the "opacity". Castor refers to $\rho \, \kappa = n \, \sigma$ is the "absorptivity". Note that the $n$ here is the number density *of the relevant population*, whereas $\rho$ is the *mass density* (e.g. of the flow at the point under consideration). 

The relationship between $\kappa$ and $\sigma$ (in the observer frame) is thus

$$\kappa_{obs} = \frac{n_{a,obs} \sigma}{\rho_{obs}}$$

The macro atom routines use $\kappa_{bf}$, which is calculated as "kap_bf" in the routine "kappa_bf". Although, actually, as near as I can tell, they don't use a "true" $\kappa$, but instead $kap\_bf = n \, \sigma$. Just to get the notation as clear as I can between my notes and the code, I'll *define* $\kappa^{\prime}$ :


$$\kappa_{obs}^\prime = \kappa_{obs} \, \, \rho_{obs}.$$

The basic photo-ionization rate estimator in the code is then called $\Gamma$ -- or explicitly "mplasma->gamma" (note that there are additionally several variants of this and the corresponding recombination estimators, to do with stimulated emission and energy weightings), defined in the code via (lines 170, 171 and 177 of estimators.c). With the notation defined here, this is

$$\Gamma = \frac{1}{\Delta h\, V_{obs}} \sum_i^{\Delta V_{obs}\Delta t_{obs}}\frac{w_{obs,i}}{\nu_{obs,i}} \kappa^{\prime}_{obs}(\nu_{cmf,i})   \delta s_{obs,i}
$$

Note that the normalization by $V_{obs}$ and $h$ (which I've intentionally taken outside the sum here for that reason) takes place outside the loop in "normalise_macro_estimators".

Note that this is *exactly* the quantity we've so far called

$$\substack{\text{ionizations}\\\\\text{per unit}\\\text{invariant space-time volume},}$$

modulo our new frame transformation factor $\left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs,i})}{c} \right]$, which of course didn't "exist" in the context of the old version of the code, where we didn't care about these effects.


## The photoionization rate estimator for simple atoms

### Background

For simple atoms, we use different photoionization rate estimators. Actually, strictly speaking, we do have a mode in which (I think) we use effectively the same estimators as for macro atoms -- this is "matrix_est" mode. However, usually we use "matrix_pow" mode, in which we don't calculate estimators directly from individual photon bundles, but instead use the photon bundles to update a banded model spectrum in each cell (i.e. we break the frequency range up into distinct bands and assume that the SED in each band can be described by a simple model, such as a power law).

The estimators we use to calculate the spectral models are updated in "update_banded_estimators" within "radiation.c". 

It helps here to remind ourselves of the relationship between the mean intensity and the properties of cells and photon bundles (e.g. Eq 23 in LK0 or Eq 97 in NS19). I'll write this down in observer frame terms for now:

$$ J_{obs,phot} = \frac{1}{4\pi V_{obs}} \sum_i^{\Delta V_{obs}\Delta t_{obs}} w_{obs,i} \, \, \delta s_{obs,i} $$ 

Here I've used the subscript "phot" to emphasise that this is a "photon-based" estimator for $J$. For the banded-model approach, $J$ is worked out separately for each band, of course.

So $J_{obs,phot}$ is updated on a photon-by-photon basis in update_banded_estimators. The same routine also works out things like the average frequency in each band. (This isn't irrelevant for frame transformation issues, because obviously here, too, one needs to think about which frame we're in.)

Without going into unnecessary details, the point of the banded-model approach is to improve the statistics by assuming a particular shape for the SED in a given cell. If the model *shape* has free parameters, we can estimate these from various statistics of the photon population (e.g. the variance). But, with the shape fixed, the *normalization* in each band is essentially determined by insisting that:

$$ J_{obs,phot} = J_{obs,mod},$$

where $J_{obs,mod}$ is the *model* prediction for the mean intensity in this cell and this band.

### Implementation

The photo-ionization rate estimator used for ionization state calculations is actually worked out in the routine "calc_pi_rate". This, in turn, uses qromb to integrate the function defined in (for example, for the power-law case) "tb_logpow". The integration then spits out

$$\int_\nu \sigma \,\, \frac{J_\nu}{\nu} d\nu .$$

Then, at the end of "calc_pi_rate", the above is normalized, giving:

$$\gamma_{pi} \,\, = \,\, 4\,\pi \int_\nu \sigma \,\, \frac{J_\nu}{h\nu} d\nu.$$

As shown, this quantity is $\gamma_{pi}$ as defined in NS19, Eq 101. It's the  photo-ionization rate per second per relevant atom (i.e. when multiplied by the number density of relevant atoms, it's the photo-ionization rate per unit time per unit volume per photo-ionization target atom). This means footnote 35 in NS19 is wrong -- the quantity defined there isn't $\gamma_{pi}$, but $n_{a} \gamma_{pi}$. Note also that they actually simply use $\gamma$ for this quantity, but I'm not here because we're already using $\gamma$ for the Lorentz factor.
So, in our notation from before, we have 

$$\Gamma = n_{a} \gamma_{pi}.$$

Now, let's think about frames. At the moment, our spectral models are defined based on weights and path lengths defined in the observer frame. I think the easiest way to make everything work out correctly is to include our SR-correction factor

\begin{equation}
K_{obs} = \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right] 
\tag{15}
\end{equation}

when we calculate our estimate of $J_{obs,phot}$ for a given cell and band. I guess the resulting quantity isn't really "observer frame" anymore then, so I'll just call it $J_{phot}$:

$$ J_{phot} = \frac{1}{4\pi V_{obs}} \sum_i^{\Delta V_{obs}\Delta t_{obs}} w_{obs,i} \, \, \delta s_{obs,i} \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right] $$ 

I think we'd similarly have to potentially include transformations for the population-based quantities we use to decide between model and fix the model parameters (e.g. power-law slopes).

Having done this, we would then calculated $\gamma_{pi}$ as before. If we do that, $\gamma_{pi}$ is also a mixed-frame sort of quantity, but one we have to multiply by observer frame $n_a$. I.e. 

$$\Gamma = n_{a, obs} \gamma_{pi}$$

## The mean intensity

At various points, we've discussed the mean intensity - so let's make this a specific example. We want this to be in the co-moving frame, so let's just write the estimator in that frame. 

$$ J_{cmf} = \frac{1}{4\pi \Delta V_{cmf}\Delta t_{cmf}} \sum_i^{\Delta V_{cmf}\Delta t_{cmf}} \epsilon_{cmf,i} \, \, \delta s_{cmf,i} $$ 

This is equation (97) in Noebauer & Sim's review paper, written with CMF subscripts. Note that I have included a time interval in the denominator which Christian doesn't have in the above (but I think probably should be there). Please also consult Christian's note below on grids and photon bundles. I've also written the weight as 
$\epsilon_{cmf,i}$, trying to make explicit that we are treating it as an energy and transforming it as such. 

Currently in the belfast20 version of python (as of 14 Aug 2020) the actual equation we calculate is 
$$ J = \frac{1}{4\pi \Delta V_{cmf} \Delta t_{obs}} \sum_i^{\Delta V_{cmf}\Delta t_{cmf}} \epsilon_{cmf,i} \, \, \delta s_{cmf,i} $$ 
where the $\Delta t_{obs}$ is implicit, because it is equal to 1 second. This is therefore missing a factor $1/\gamma$ to convert $\Delta t_{obs}$ to $\Delta t_{cmf}$. 

***ksl: This has been fixed in normalize_simple_estimators.  Please confirm***

# Metaphysics

Stuart raised a couple of "conceptual" issues/worries in his notes that I wanted to touch on, now that I've had a chance to think about all this.


## On grids and photon bundles

Since our computational grid is defined in the observer frame, we don't really know what $\Delta V_{cmf}$ is. All we really know is what $\Delta V_{obs}$ is. We do know that $\Delta V_{cmf} \Delta t_{cmf} = \Delta V_{obs} \Delta t_{obs}$, but Stuart's worry, I believe, was that the we can't be sure we are talking about the *same* space-time volume element in both frames, really, and therefore the photon bundles we are talking about are also not the same. 

In my "derivation" above, I've argued explicitly that we can simply *define* our comoving space-time volume element to be that which contains the photon bundles we find in the observer-frame space-time volume element. If this is correct, that worry disappears.

Is it correct though? I can't see any reason why it would not be correct, except that our space-time volume elements aren't really infinitesimal. So the equality between the elements, and that photon bundles they contain, may not be exact. However, at that level, I don't think this issue is any different than the issue of selecting a suitable coordinate system (i.e. spherical polar vs Cartesian) or computational grid (i.e. 50x50 vs 100x100) for a calculation. When we do that, we will end up with (hopefully) slightly different values for all estimators at a given point in space, simply because our cells are now different. So photons that in one system/grid contribute to one cell will contribute to a different cell in a different system/grid. But **so long as the cells are small** and **so long as we use the correct (space)time volume for them**, these differences should not be significant, and they should vanish as our cells get smaller and smaller. For pathologically shaped cells (in any one system/grid), that conversion may be slower than we'd like. But it should still happen. 
    
##  Is the comoving frame actually inertial?

Stuart also notes the concern that, strictly speaking, the comoving frame isn't inertial. Really. the flow is accelerating, so is it legitimate to use *Lorentz* transformations -- which are defined explicitly for inertial frames? 

This is actually also noted and briefly discussed by Mihalas (mihalas-1; p144). Effectively, the answer is "treating each point in the flow as instantaneously inertial yields an internally consistent picture that's consistent with experiment". And if it's good enough for Mihalas....

## Another example:  bound-bound wind emission

We assume we know the temperature of a cell in the comoving frame (because we've balanced heating and cooling rates in consistent frames). We've also worked out all of the ionization and excitation balances, so we know all level populations and hence comoving frame densities. In the comoving frame, where emission actually takes place, we can therefore write the line luminosity of a single line -- rest frequency $\nu_0$ -- in the usual way:

\begin{equation}
L_{line,cmf} = n_{ul,cmf} A_{ul} (h\nu_0) \Delta V_{cmf}
\tag{18}
\end{equation}

Let's first of all transform this again into things we already know in the code right now. Since $n_{ul,cmf} \Delta V_{cmf}$ is a pure number (the number of atoms in the cell), it's invariant, i.e. $n_{ul,cmf} \Delta V_{cmf} = n_{ul,obs} \Delta V_{obs}$


\begin{equation}
L_{line,cmf} = n_{ul,obs} A_{ul} (h\nu_0) \Delta V_{obs}
\tag{19}
\end{equation}

OK, so we now how to generate the correct line luminosity in the cmf. Now what?

**JM note: I think the below may be a little at odds with what we decided to do, and how we are deciding to think of "weight". It should be updated to reflect this.** 

Here are the concrete things we would then need to do, I think, in order to end up with a correct photon distribution in the observer frame:

<br>

* We know we want to generate $L_{line,cmf}$ in the cell 

<br>

* So we draw $N_{\gamma}$ photons from/with the appropriate distribution in the cmf. In the simplest case, this would be (just as an example):

    * all with $\nu_0$ (e.g. if we ignored thermal broadening)
    * isotropic (if we ignored the direction dependence of the resonance zone escape probability)

<br>

* We can decide how many photons we want to generate pretty much arbitrarily -- what we care about is just that the total luminosity is correct. Let's say we generate $N_\gamma$ of them.

<br>

* In *Python*, weight = luminosity, so, in the comoving each photon there carries cmf weight 

\begin{equation}
w_{cmf} = L_{line,cmf} / N_{\gamma}
\tag{20}
\end{equation}

<br>

* So we generate all $N_\gamma$ photons as described above, i.e. each has the following properties (for our simple example)

    * cmf direction chosen from an isotropic distribution
    * cmf weight given by Equation 20
    * cmf frequency given by $\nu_0$

<br>

* We now need to transform each of these photon properties into the observer frame
    * observer direction follows from the angle aberration transformation
    * frequency has dimensions of 1/time, so transforms as the inverse of Equation 12
\begin{equation}
\delta t_{cmf} = \gamma \delta t_{obs} \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right]
\tag{21}
\end{equation}
\begin{equation}
\nu_{obs} = \gamma \nu_{cmf} \left[ 1 - \frac{(\mathbf{v} \cdot \mathbf{\hat{n}}_{obs})}{c} \right]
\tag{22}
\end{equation}
    * weights transform as

\begin{equation}
w_{obs} = \frac{h\nu_{obs}}{h\nu_{cmf}} \frac{\Delta t_{cmf}}{\Delta t_{obs}} w_{cmf}
\tag{23}
\end{equation}

<br>

* Pure time intervals (clock ticks) transform as 
\begin{equation}
\Delta t_{obs} = \gamma \Delta t_{cmf}
\tag{23}
\end{equation}
and therefore the $\gamma$ cancel, leaving
\begin{equation}
w_{obs} = \left(1 - \frac{\bf{n} \cdot \bf{v}}{c}\right) w_{cmf}
\tag{23}
\end{equation}

<br>

* Note that this weight transformation has to be applied *only* for emission, not for scattering